
# Setting Up Enviroment



In [2]:
!pip install syft

     |████████████████████████████████| 256kB 8.9MB/s 
     |████████████████████████████████| 204kB 51.7MB/s 
     |████████████████████████████████| 460kB 56.7MB/s 
     |████████████████████████████████| 81kB 30.4MB/s 
     |████████████████████████████████| 256kB 63.6MB/s 
     |████████████████████████████████| 2.1MB 59.1MB/s 
     |████████████████████████████████| 389kB 45.5MB/s 
     |████████████████████████████████| 51kB 23.5MB/s 
     |████████████████████████████████| 266kB 66.7MB/s 
     |████████████████████████████████| 122kB 57.3MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067055 sha256=9d9d2923cd7b767d6fa5ff029f1a857356f4db02cfe04b56a14d43d855719d0d
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=5a61ed57c2b1ea08a28b8551a1e59062165a7ee43c29efe515253200b4322cc3
  Stored in di

In [1]:
import syft as sy

import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms


W0818 07:40:51.933350 140371010914176 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0818 07:40:51.947892 140371010914176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bs = 1024

# Define a transform to normalize the data
tfms = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,))])

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=tfms)

trainloader = th.utils.data.DataLoader(mnist_trainset, batch_size=bs, shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [65]:
# Use GPU if it's available
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cuda')

# Model and Data

In [0]:
model = Net()

# model = model.to(device)

In [0]:
criterion = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01) 
# TODO momentum is not supported at the moment
# optimizer = optim.Adam(model.parameters(), lr=0.003)

In [0]:
epochs = 15
steps = 0
running_loss = 0

In [69]:
for epoch in range(epochs):
  
  model.train()
  
  for inputs, labels in trainloader:
#     inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    logits = model(inputs)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    
  print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss:.3f}")
  
  running_loss = 0

Epoch 1/15.. Train loss: 135.624
Epoch 2/15.. Train loss: 134.414
Epoch 3/15.. Train loss: 132.984
Epoch 4/15.. Train loss: 130.911
Epoch 5/15.. Train loss: 127.550
Epoch 6/15.. Train loss: 121.836
Epoch 7/15.. Train loss: 112.131
Epoch 8/15.. Train loss: 97.625
Epoch 9/15.. Train loss: 81.515
Epoch 10/15.. Train loss: 68.254
Epoch 11/15.. Train loss: 58.366
Epoch 12/15.. Train loss: 50.841
Epoch 13/15.. Train loss: 45.102
Epoch 14/15.. Train loss: 40.807
Epoch 15/15.. Train loss: 37.581


# Federated Model and Data

In [37]:
device.type == 'cpu'

False

In [0]:
hook = sy.TorchHook(th)

In [15]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0818 07:47:44.043342 140371010914176 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0818 07:47:44.045347 140371010914176 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0818 07:47:44.046973 140371010914176 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [38]:
kwargs = {'num_workers': 4, 'pin_memory': True} if (device.type == 'cuda') else {}

federated_dataset = mnist_trainset.federate((bob, alice, secure_worker))

federated_trainloader = sy.FederatedDataLoader(federated_dataset, batch_size=bs, shuffle=True, **kwargs)

W0818 08:45:22.602704 140371010914176 dataloader.py:197] The following options are not supported: num_workers: 4, pin_memory: True


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [0]:
model = Net()

# model = Net().to(device) 
# Expected object of backend CPU but got backend CUDA for argument #2 'source'

In [0]:
criterion = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01) 
# TODO momentum is not supported at the moment
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [0]:
epochs = 15
steps = 0
running_loss = 0

In [59]:
for epoch in range(epochs):
  
  model.train()
  
  for batch_idx, (inputs, labels) in enumerate(federated_trainloader):
    
    model.send(inputs.location)

    optimizer.zero_grad()

    logps = model(inputs)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    model = model.get()

    running_loss += loss.get().item()
    
  print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss:.3f}")
  
  running_loss = 0

Epoch 1/15.. Train loss: 137.387
Epoch 2/15.. Train loss: 135.923
Epoch 3/15.. Train loss: 133.928
Epoch 4/15.. Train loss: 130.809
Epoch 5/15.. Train loss: 125.564
Epoch 6/15.. Train loss: 116.430
Epoch 7/15.. Train loss: 101.872
Epoch 8/15.. Train loss: 84.948
Epoch 9/15.. Train loss: 70.821
Epoch 10/15.. Train loss: 60.593
Epoch 11/15.. Train loss: 53.165
Epoch 12/15.. Train loss: 47.339
Epoch 13/15.. Train loss: 42.834
Epoch 14/15.. Train loss: 39.410
Epoch 15/15.. Train loss: 36.827


In [0]:
sy.FederatedDataLoader?
